In [1]:
import tensorflow as tf
import numpy as np

def generate_sample_data(num_data):
    X = np.random.rand(num_data, 1)
    Y = X * 2 + np.random.normal(3, .1, [num_data,1])
    return X, Y

In [2]:
train_X, train_Y = generate_sample_data(30)
test_X, test_Y = generate_sample_data(10)

# Train and save model

In [3]:
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

W = tf.Variable(np.random.randn(), dtype=tf.float32, name='weight')
b = tf.Variable(np.random.randn(), dtype=tf.float32, name='bias')

another_variable = tf.Variable(np.random.randn(), name='another_var')

pred = tf.add(tf.multiply(X, W), b)
cost = tf.reduce_mean(tf.pow(pred-Y, 2), name='cost')
optimizer = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(cost)

init = tf.global_variables_initializer()

n_epoches = 10
display_epoch = 3
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epoches):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y:y})
        if (epoch) % display_epoch == 0:
            c_tr = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            c_te = sess.run(cost, feed_dict={X: test_X, Y:test_Y})
            print("Epoch={}, cost-tr={:.3f}, cost-te={:.3f}, W={:.3f}, b={:.3f}".format(epoch, c_tr, c_te, W.eval(), b.eval()))

    print("Optimization done")
    testing_cost = sess.run(cost, feed_dict={X: test_X, Y: test_Y})
    print("Testing cost: {:.3f}, with w={:.3f}, b={:.3f}".format(testing_cost, W.eval(), b.eval()))
    
    # save model
    saver = tf.train.Saver()
    save_path = saver.save(sess, './saved_models/linear_regression')
    print("Model saved in file: %s" % save_path)

Epoch=0, cost-tr=0.584, cost-te=0.578, W=2.523, b=2.012
Epoch=3, cost-tr=0.099, cost-te=0.080, W=2.803, b=2.459
Epoch=6, cost-tr=0.073, cost-te=0.055, W=2.754, b=2.572
Epoch=9, cost-tr=0.060, cost-te=0.043, W=2.670, b=2.632
Optimization done
Testing cost: 0.043, with w=2.670, b=2.632
Model saved in file: ./saved_models/linear_regression


# Clear environment and load model
# Restart the kernel first

In [1]:
import tensorflow as tf
import numpy as np

def generate_sample_data(num_data):
    X = np.random.rand(num_data, 1)
    Y = X * 2 + np.random.normal(3, .1, [num_data,1])
    return X, Y

text_X, text_Y = generate_sample_data(5)

In [2]:
tf.reset_default_graph()

sess = tf.Session()
new_saver = tf.train.import_meta_graph(
    './saved_models/linear_regression.meta')
new_saver.restore(
    sess, tf.train.latest_checkpoint('./saved_models'))

# no need to initialize vars
# init = tf.global_variables_initializer()
# sess.run(init)

all_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
for v in all_vars:
    print(v.name)
    v_ = sess.run(v)
    print(v.name, ' | ', v_)
    
# print variables of interest
print('\n')
print('print values we have interest')
print('weight: ', sess.run('weight:0'))
print('bias: ', sess.run('bias:0'))
    
# use loaded model for prediction
print('\n')
graph = tf.get_default_graph()
X = graph.get_tensor_by_name('X:0')
Y = graph.get_tensor_by_name('Y:0')
cost = graph.get_tensor_by_name('cost:0')
feed_dict = {X: text_X, Y:text_Y}
cost_value = sess.run(cost, feed_dict)

print('cost', cost_value)

INFO:tensorflow:Restoring parameters from ./saved_models/linear_regression
weight:0
weight:0  |  2.67017
bias:0
bias:0  |  2.63157
another_var:0
another_var:0  |  0.659564
weight/Adagrad:0
weight/Adagrad:0  |  67.9041
bias/Adagrad:0
bias/Adagrad:0  |  301.993


print values we have interest
weight:  2.67017
bias:  2.63157


cost 0.0682534
